
1. How many copies of the film _Hunchback Impossible_ exist in the inventory system?
2. List all films whose length is longer than the average of all the films.
3. Use subqueries to display all actors who appear in the film _Alone Trip_.
4. Sales have been lagging among young families, and you wish to target all family movies for a promotion. Identify all movies categorized as family films.
5. Get name and email from customers from Canada using subqueries. Do the same with joins. Note that to create a join, you will have to identify the correct tables with their primary keys and foreign keys, that will help you get the relevant information.
6. Which are films starred by the most prolific actor? Most prolific actor is defined as the actor that has acted in the most number of films. First you will have to find the most prolific actor and then use that actor_id to find the different films that he/she starred.
7. Films rented by most profitable customer. You can use the customer table and payment table to find the most profitable customer ie the customer that has made the largest sum of payments
8. Customers who spent more than the average payments.

<img src="Labs/img/sakila_db_image.png" alt="Drawing" style="width: 100px;"/>

In [6]:
#setup
import pandas as pd
import getpass
import sqlalchemy as sa

#### connect

In [7]:
driver = 'mysql+pymysql'
user = 'root'
password = getpass.getpass(prompt='Your password:'"")
ip = 'localhost' #'127.0.0.1'  

In [8]:
connection_string = f'{driver}://{user}:{password}@{ip}'
db_con = sa.create_engine(connection_string)

### 1. How many copies of the film _Hunchback Impossible_ exist in the inventory system?

In [9]:
db_con.execute("use sakila;")

In [10]:
films ="SELECT * FROM sakila.film"
film = pd.read_sql_query(films, db_con)
film.head(1)

,film_id,title,description,release_year,language_id,original_language_id,rental_duration,rental_rate,length,replacement_cost,rating,special_features,last_update
0,1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,2006,1,None,6,0.99,86,20.99,PG,"Deleted Scenes,Behind the Scenes",2006-02-15 05:03:42


In [11]:
res_str = "select * from sakila.inventory"
res = pd.read_sql(res_str, db_con)

In [12]:
res.head()

,inventory_id,film_id,store_id,last_update
0,1,1,1,2006-02-15 05:09:17
1,2,1,1,2006-02-15 05:09:17
2,3,1,1,2006-02-15 05:09:17
3,4,1,1,2006-02-15 05:09:17
4,5,1,2,2006-02-15 05:09:17


In [13]:
# merge film and inventory tables on key 'film_id'
inv_film = film.merge(res, left_on="film_id", right_on="film_id")

In [14]:
# find the right movie
hunch_movie = inv_film[inv_film["title"].str.startswith("HUNCH")]

### ***answer:***

In [15]:
print('Movie:',hunch_movie.title.unique(),'exist in inventory for',hunch_movie.film_id.count(),'times')
hunch_movie

Movie: ['HUNCHBACK IMPOSSIBLE'] exist in inventory for 6 times


,film_id,title,description,release_year,language_id,original_language_id,rental_duration,rental_rate,length,replacement_cost,rating,special_features,last_update_x,inventory_id,store_id,last_update_y
2016,439,HUNCHBACK IMPOSSIBLE,A Touching Yarn of a Frisbee And a Dentist who...,2006,1,None,4,4.99,151,28.99,PG-13,"Trailers,Deleted Scenes",2006-02-15 05:03:42,2017,1,2006-02-15 05:09:17
2017,439,HUNCHBACK IMPOSSIBLE,A Touching Yarn of a Frisbee And a Dentist who...,2006,1,None,4,4.99,151,28.99,PG-13,"Trailers,Deleted Scenes",2006-02-15 05:03:42,2018,1,2006-02-15 05:09:17
2018,439,HUNCHBACK IMPOSSIBLE,A Touching Yarn of a Frisbee And a Dentist who...,2006,1,None,4,4.99,151,28.99,PG-13,"Trailers,Deleted Scenes",2006-02-15 05:03:42,2019,1,2006-02-15 05:09:17
2019,439,HUNCHBACK IMPOSSIBLE,A Touching Yarn of a Frisbee And a Dentist who...,2006,1,None,4,4.99,151,28.99,PG-13,"Trailers,Deleted Scenes",2006-02-15 05:03:42,2020,1,2006-02-15 05:09:17
2020,439,HUNCHBACK IMPOSSIBLE,A Touching Yarn of a Frisbee And a Dentist who...,2006,1,None,4,4.99,151,28.99,PG-13,"Trailers,Deleted Scenes",2006-02-15 05:03:42,2021,2,2006-02-15 05:09:17
2021,439,HUNCHBACK IMPOSSIBLE,A Touching Yarn of a Frisbee And a Dentist who...,2006,1,None,4,4.99,151,28.99,PG-13,"Trailers,Deleted Scenes",2006-02-15 05:03:42,2022,2,2006-02-15 05:09:17


### 2. List all films whose length is longer than the average of all the films.

In [16]:
# need all film and lenght
# calculate average of all movies
# use greater than check to find the right movies

movie_query = "select title, length from film"
res = pd.read_sql(movie_query, db_con)

In [17]:
data = res[res['length'] > res['length'].mean()]

In [18]:
# check if result makes sense
res.describe() # mean by 115.27

,length
count,1000.000000
mean,115.272000
std,40.426332
min,46.000000
25%,80.000000
50%,114.000000
75%,149.250000
max,185.000000


In [19]:
data.sort_values(['length'], ascending=False)

,title,length
181,CONTROL ANTHEM,185
140,CHICAGO NORTH,185
608,MUSCLE BRIGHT,185
871,SWEET BROTHERHOOD,185
816,SOLDIERS EVOLUTION,185
...,...,...
3,AFFAIR PREJUDICE,117
986,WORDS HUNTER,116
542,MADIGAN DORADO,116
462,INSTINCT AIRPORT,116


### 3. Use subqueries to display all actors who appear in the film _Alone Trip_.

In [20]:
# query film Alone trip
# query actors 
# 'film actor' holds key for actor_id and film_id

# 1 get film_id

In [21]:
res = """select concat(first_name,' ',last_name) as Actor_name from sakila.actor 
where 
    actor_id in (
        select actor_id from sakila.film_actor 
            inner join 
                sakila.film 
            using (film_id) 
                where film_id = 
                    (select film_id from sakila.film
                        where title = 'Alone trip'
                    )
                );"""

In [22]:
pd.read_sql(res, db_con)

,Actor_name
0,ED CHASE
1,KARL BERRY
2,UMA WOOD
3,WOODY JOLIE
4,SPENCER DEPP
5,CHRIS DEPP
6,LAURENCE BULLOCK
7,RENEE BALL


### 4. Sales have been lagging among young families, and you wish to target all family movies for a promotion. Identify all movies categorized as family films.

In [23]:
query = """select title from sakila.film
where film_id in 
                (select film_id from sakila.film_category 
                    where category_id = 
                        (select category_id from sakila.category
                            where name = 'Family'
                        )
                );"""

In [24]:
pd.read_sql(query, db_con)

,title
0,AFRICAN EGG
1,APACHE DIVINE
2,ATLANTIS CAUSE
3,BAKED CLEOPATRA
4,BANG KWAI
...,...
64,SPLASH GUMP
65,SUNSET RACER
66,SUPER WYOMING
67,VIRTUAL SPOILERS


### 5. Get name and email from customers from Canada using subqueries. Do the same with joins. Note that to create a join, you will have to identify the correct tables with their primary keys and foreign keys, that will help you get the relevant information.

In [25]:
# first using subqueries

In [26]:
query = """
select concat(first_name,' ',last_name) as Name, email from customer where address_id in (
select address_id from address where city_id in (
select city_id from city where country_id = (
select country_id from country where country = 'canada')));
"""

In [27]:
pd.read_sql(query, db_con)

,Name,email
0,DERRICK BOURQUE,DERRICK.BOURQUE@sakilacustomer.org
1,DARRELL POWER,DARRELL.POWER@sakilacustomer.org
2,LORETTA CARPENTER,LORETTA.CARPENTER@sakilacustomer.org
3,CURTIS IRBY,CURTIS.IRBY@sakilacustomer.org
4,TROY QUIGLEY,TROY.QUIGLEY@sakilacustomer.org


In [28]:
# second using joints

In [29]:
query = '''
select concat(first_name,' ',last_name) as Name ,email from city 
inner join 
	country
using (country_id)
inner join
	address
using (city_id)
inner join
	customer
using (address_id)
where country = 'canada';

'''

In [30]:
pd.read_sql(query, db_con)

,Name,email
0,DERRICK BOURQUE,DERRICK.BOURQUE@sakilacustomer.org
1,DARRELL POWER,DARRELL.POWER@sakilacustomer.org
2,LORETTA CARPENTER,LORETTA.CARPENTER@sakilacustomer.org
3,CURTIS IRBY,CURTIS.IRBY@sakilacustomer.org
4,TROY QUIGLEY,TROY.QUIGLEY@sakilacustomer.org


### 6. Which are films starred by the most prolific actor? Most prolific actor is defined as the actor that has acted in the most number of films. First you will have to find the most prolific actor and then use that actor_id to find the different films that he/she starred.

In [31]:
query = """
select title as Movie_name from film
where 
    film_id in (
        select film_id from film_actor 
            where actor_id = (
                select actor_id as movie_star from sakila.film_actor
                    group by 
                        actor_id 
                    order by 
                        count(actor_id) desc limit 1));
"""

In [32]:
pd.read_sql(query, db_con)

,Movie_name
0,BED HIGHBALL
1,CALENDAR GUNFIGHT
2,CHAMBER ITALIAN
3,CHAPLIN LICENSE
4,CHARIOTS CONSPIRACY
5,CLUELESS BUCKET
6,COLDBLOODED DARLING
7,CONEHEADS SMOOCHY
8,DARKNESS WAR
9,DEER VIRGINIAN


### 7. Films rented by most profitable customer. You can use the customer table and payment table to find the most profitable customer ie the customer that has made the largest sum of payments

In [33]:
query = """

select title from film
where film_id in (
select film_id from inventory
where inventory_id in (

select inventory_id from rental 
where
	customer_id = (
select customer_id from payment
group by
	customer_id
order by
	sum(amount) desc limit 1)));

"""

In [34]:
pd.read_sql(query, db_con)

,title
0,DESTINY SATURDAY
1,CYCLONE FAMILY
2,SLUMS DUCK
3,FIDELITY DEVIL
4,SPLASH GUMP
5,MISSION ZOOLANDER
6,MULHOLLAND BEAST
7,PRINCESS GIANT
8,PARIS WEEKEND
9,RACER EGG


### 8. Customers who spent more than the average payments.

In [75]:
# check function for test
query_check = """select @mean_var;"""

# set mean variable
query_mean = """
set @mean_var =(select avg(mean) from
( 
select avg(amount) as mean 
	from 
		sakila.payment
	group by
		customer_id
)sub1);
"""

# run query to find all customer
query_customer = """
select customer_id from sakila.payment
group by
	customer_id	
having
	avg(amount) > (select @mean_var)
"""


# combine string to fetch names from id
query_names = """
select concat(first_name,' ', last_name) as name from sakila.customer
where customer_id in (""" + str(query_customer)+ """);"""


check mean value

In [76]:
# set mean value on database site
db_con.execute(query_mean)

# get calculated mean value
res = db_con.execute(query_check)
for mean_value in res:
    print(mean_value)

(Decimal('4.197934931552587646'),)


In [73]:
# get customer id numbers

pd.read_sql(query_customer, db_con)

,customer_id
0,2
1,3
2,7
3,11
4,13
...,...
295,590
296,591
297,593
298,594


In [77]:
# get names from customers which spend more than average

pd.read_sql(query_names, db_con)

,name
0,PATRICIA JOHNSON
1,LINDA WILLIAMS
2,MARIA MILLER
3,LISA ANDERSON
4,KAREN JACKSON
...,...
295,SETH HANNON
296,KENT ARSENAULT
297,RENE MCALISTER
298,EDUARDO HIATT
